In [ ]:
!pip install pycoingecko requests tiktoken cohere openai langchain langchain_experimental google-search-results

###1.IMPORT LIBRARY

In [ ]:
import os
import requests
import json
from datetime import datetime

from langchain.document_loaders.base import Document
from langchain.utilities import ApifyWrapper
from langchain.vectorstores import Vectara
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains import RetrievalQA
from langchain.schema.output_parser import StrOutputParser
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate,ChatPromptTemplate, MessagesPlaceholder

###2.KEY SETTINGS

In [ ]:
os.environ["OPENAI_API_KEY"]      = "OPENAI_API_KEY"
os.environ["VECTARA_CUSTOMER_ID"] = "VECTARA_CUSTOMER_ID"
os.environ["VECTARA_CORPUS_ID"]   = "VECTARA_CORPUS_ID"
os.environ["VECTARA_API_KEY"]     = "VECTARA_API_KEY"
os.environ["GOOGLE_API_KEY"]      = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"]       = "GOOGLE_CSE_ID"

###3.INITIALIZE VECTARA

In [ ]:
vectara = Vectara(
    vectara_customer_id = os.getenv("VECTARA_CUSTOMER_ID"),
    vectara_corpus_id   = os.getenv("VECTARA_CORPUS_ID"),
    vectara_api_key     = os.getenv("VECTARA_API_KEY")
)

In [ ]:
# ChatGPT 모델 지정
#llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)
llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

In [ ]:
# Create Prompt
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
{context}
Question: {question}
Relevant text, if any:"""
#prompt = ChatPromptTemplate.from_template(question_prompt_template)
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

#chain_type_kwargs = {"verbose": True,  "question_prompt": QUESTION_PROMPT}

In [ ]:
# Define retriever
retriever = vectara.as_retriever(lambda_val=0.025, k=5, filter=None)

from langchain.chains import RetrievalQAWithSourcesChain

knowledgeBase_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    input_key="query",
    chain_type_kwargs={
        "prompt": QUESTION_PROMPT
    },
)

In [ ]:
query = "지식기반 기준으로 답해주세요.시스템반도체산업 현황 및 전망"
response = knowledgeBase_qa({"query": query},return_only_outputs=True)

In [ ]:
source_documents = response['source_documents']

# 결과...
## LLM Result
## Metadata - 포멧 이쁘게 해서 보여주세요

In [ ]:
print(response['result'])

for i in range(len(source_documents)):
  #print(source_documents[i].page_content[0:100])
  print(source_documents[i].metadata)